In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

CONTEXT_SIZE = 2
EMBEDDING_DIM = 50

input_text = """Development of a Future Risk Assessment Model for Financing Applicants
This document details the development of a future risk assessment model for financing applicants, 
commissioned by Blank Bank from Tripleedata.com. The document outlines the specific procedures employed 
and the results obtained throughout the model development process. Key steps and parameters are highlighted, including 
sample design, exclusion criteria, and the model development methodologies utilized. Additionally, relevant tables and 
reports are incorporated to summarize the findings."""
test_sentence = input_text.split()

# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(f'the loss is:{losses}')  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "outlines"
print(model.embeddings.weight[word_to_ix["outlines"]])

[(['of', 'Development'], 'a'), (['a', 'of'], 'Future'), (['Future', 'a'], 'Risk')]
the loss is:[294.75642561912537, 291.99520921707153, 289.2646527290344, 286.5657856464386, 283.88757252693176, 281.2272951602936, 278.57863116264343, 275.9407570362091, 273.31093740463257, 270.69440722465515]
tensor([ 0.3000, -0.1527, -1.5835, -0.6231,  1.3041,  0.4765, -0.8786, -1.6595,
         0.5570,  1.2145,  0.1724,  0.1509,  1.5209,  0.2957,  0.6320, -0.2658,
        -0.2190, -0.9276,  0.1965, -0.7335,  0.8411, -0.1541, -0.0806,  0.2923,
        -0.4367,  0.3356,  0.5590,  2.3130,  0.0489,  0.2435,  1.6229,  0.1675,
        -0.7811,  1.0074,  1.6040, -1.3267, -0.4023,  1.6042,  0.4359, -0.1282,
        -1.0770,  1.1537,  0.8967, -0.6429, -1.1614, -0.8552,  0.3006,  0.2418,
        -0.8400, -0.9485], grad_fn=<SelectBackward0>)


In [ ]:
## End